<a href="https://colab.research.google.com/github/thenaivekid/Video-gen-with-voice-cloning-and-deep-fake/blob/main/semicolon_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ## ashok https://res.cloudinary.com/dbhckg5el/video/upload/v1735888507/xaek5lcx3siz7v8ufgiy.m4a
# # {
#   "text": "I am the guy who does work",
#   "file_name": "temp",
#   "source_audio_url": "https://drive.google.com/file/d/1x4PjhXvh-78H7CcwdWPNdda3bd30sTHu/view?usp=sharing"
# }

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [ ]:
!conda create -n tts
!source activate tts && pip install TTS pyngrok fastapi uvicorn cloudinary gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os
os.environ["NGROK_AUTHTOKEN"] = "2ou2c1Bii8BIoxgpVvLbyrGRrhq_39atbrbFQZoZsf6o7MYsQ"

In [ ]:
%%writefile test.py
from fastapi import FastAPI, Form
from pydantic import BaseModel
from TTS.api import TTS
from pyngrok import ngrok
import os
import requests
import tempfile
import cloudinary
import cloudinary.uploader
import gdown

# Initialize FastAPI app
app = FastAPI()

# Initialize TTS model
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# Configure Cloudinary
cloudinary.config(
    cloud_name="dbhckg5el",
    api_key="592613464587995",
    api_secret="uk1Ackb_FnPqLbX12ojf65Izct8",
    secure=True
)

class TTSRequest(BaseModel):
    text: str
    file_name: str
    source_audio_url: str

@app.post("/tts")
def generate_tts(req: TTSRequest):
    try:
        # Determine if the URL is a Google Drive link
        if "drive.google.com" in req.source_audio_url:
            # Extract the file ID from the URL
            file_id = req.source_audio_url.split("/d/")[1].split("/" if "/" in req.source_audio_url.split("/d/")[1] else "?")[0]
            gdrive_url = f"https://drive.google.com/uc?id={file_id}"

            # Download the file using gdown
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_source_audio:
                gdown.download(gdrive_url, temp_source_audio.name, quiet=False)
                temp_source_audio_path = temp_source_audio.name
        else:
            # Download source audio from direct URL
            response = requests.get(req.source_audio_url, stream=True)
            if response.status_code != 200:
                return {"error": "Failed to download source audio."}

            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_source_audio:
                temp_source_audio.write(response.content)
                temp_source_audio_path = temp_source_audio.name

        # Generate the TTS output
        output_file_path = f"{req.file_name}.wav"
        tts.tts_to_file(
            text=req.text,
            file_path=output_file_path,
            speaker_wav=temp_source_audio_path,
            language="en"
        )

        # Upload the output file to Cloudinary
        cloudinary_response = cloudinary.uploader.upload(
            output_file_path,
            resource_type="video"  # "video" supports audio files in Cloudinary
        )

        # Clean up temporary files
        os.remove(temp_source_audio_path)
        os.remove(output_file_path)

        return {"cloudinary_url": cloudinary_response['secure_url']}
    except Exception as e:
        return {"error": str(e)}

# Run ngrok tunnel and expose FastAPI server
@app.on_event("startup")
def startup_event():
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

# Command to start the server
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


Overwriting test.py


In [ ]:
!source activate tts && python test.py

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
/usr/local/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any is

In [ ]:
# import cloudinary
# import cloudinary.uploader

# cloudinary.config(
#     cloud_name="dbhckg5el",
#     api_key="592613464587995",
#     api_secret="uk1Ackb_FnPqLbX12ojf65Izct8",
#     secure=True
# )
# def upload_to_cloudinary(file_path):
#     try:
#         response = cloudinary.uploader.upload(
#             file_path,
#             resource_type="video"  # "video" supports audio files like .wav
#         )
#         return {"cloudinary_url": response["secure_url"]}
#     except Exception as e:
#         return {"error": str(e)}

# # Example usage
# file_path = "/content/drive/MyDrive/semicolon/ashok.mp4"
# result = upload_to_cloudinary(file_path)
# print(result)


{'cloudinary_url': 'https://res.cloudinary.com/dbhckg5el/video/upload/v1735888507/xaek5lcx3siz7v8ufgiy.m4a'}
